In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
text = """
From fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou, contracted to thine own bright eyes,
Feed'st thy light's flame with self-substantial fuel,
Making a famine where abundance lies,
Thyself thy foe, to thy sweet self too cruel.
"""

In [22]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [23]:
# Padding sequences
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = tf.keras.utils.to_categorical(label, num_classes=total_words)



In [24]:
#Build LSTM model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 8, 100)            5300      
                                                                 
 lstm_3 (LSTM)               (None, 8, 150)            150600    
                                                                 
 dropout (Dropout)           (None, 8, 150)            0         
                                                                 
 lstm_4 (LSTM)               (None, 150)               180600    
                                                                 
 dense_1 (Dense)             (None, 53)                8003      
                                                                 
Total params: 344503 (1.31 MB)
Trainable params: 344503 (1.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
# Train the model
model.fit(predictors, label, epochs=500, verbose=1)

Epoch 1/500
2/2 [==============================] - 4s 33ms/step - loss: 3.9717 - accuracy: 0.0196
Epoch 2/500
2/2 [==============================] - 0s 32ms/step - loss: 3.9633 - accuracy: 0.0588
Epoch 3/500
2/2 [==============================] - 0s 30ms/step - loss: 3.9546 - accuracy: 0.0588
Epoch 4/500
2/2 [==============================] - 0s 31ms/step - loss: 3.9449 - accuracy: 0.0588
Epoch 5/500
2/2 [==============================] - 0s 30ms/step - loss: 3.9302 - accuracy: 0.0588
Epoch 6/500
2/2 [==============================] - 0s 34ms/step - loss: 3.9038 - accuracy: 0.0588
Epoch 7/500
2/2 [==============================] - 0s 30ms/step - loss: 3.8703 - accuracy: 0.0588
Epoch 8/500
2/2 [==============================] - 0s 31ms/step - loss: 3.8373 - accuracy: 0.0588
Epoch 9/500
2/2 [==============================] - 0s 32ms/step - loss: 3.8181 - accuracy: 0.0588
Epoch 10/500
2/2 [==============================] - 0s 37ms/step - loss: 3.8016 - accuracy: 0.0588
Epoch 11/500
2/2 [=

In [26]:
# Function to generate text with temperature sampling
def generate_text(seed_text, next_words, model, max_sequence_len, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

        # Predict the probabilities of each word
        predicted_probs = model.predict(token_list, verbose=0).flatten()

        # Apply temperature sampling
        predicted_probs = np.log(predicted_probs) / temperature
        predicted_probs = np.exp(predicted_probs) / np.sum(np.exp(predicted_probs))

        # Sample from the distribution
        predicted = np.random.choice(range(total_words), size=1, p=predicted_probs)[0]

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [27]:
# Generate text
seed_text = "From fairest creatures we desire"
generated_text = generate_text(seed_text, 10, model, max_sequence_len, temperature=0.5)
print(generated_text)

From fairest creatures we desire increase memory decease eyes eyes eyes eyes eyes eyes eyes
